# Machine Learning 3 - Support Vector Machines

A SVM classifier builds a set of hyper-planes to try and separate the data by maximizing the distance between the borders and the data points.

![SVM](http://scikit-learn.org/stable/_images/sphx_glr_plot_separating_hyperplane_0011.png "Decision border in an SVM")

This separation is generally not possible to achieve in the original data space. Therefore, the first step of the SVM is to project the data into a high or infinite dimensions space in which this linear separation can be done. The projection can be done with linear, polynomial, or more comonly "RBF" kernels.

In [1]:
from lab_tools import CIFAR10, evaluate_classifier, get_hog_image
import numpy as np

dataset = CIFAR10('./CIFAR10')

def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data,axis=0)
    data = (data - mean)/std
    return data, mean, std

Pre-loading training data
Pre-loading test data


**Build a simple SVM** using [the SVC (Support Vector Classfiication) from sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). 
**Train** it on the CIFAR dataset.

In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

#kf = StratifiedKFold(1, random_state=33)

total_score = 0
total_conf = 0

C = 2**3
gamma = 2**-1
#for train,test in kf.split(dataset.train['hog'], dataset.train['labels']):
svc = SVC(C=C, kernel='rbf', gamma=gamma, )
train_x, mean, std = normalize(dataset.train['hog'][:14000])
train_y = dataset.train['labels'][:14000]
svc.fit(train_x, train_y)

test_x = (dataset.train['hog'][14000:]-mean)/std
test_y = dataset.train['labels'][14000:]
accuracy, conf = evaluate_classifier(svc, test_x, test_y)

total_score = total_score + accuracy
total_conf = total_conf + conf

15000

In [7]:
total_score = total_score/5
total_conf = total_conf/5

print("Validation accuracy: %.2f %%"%total_score)
print("Confusion matrix:")
print(total_conf)

Validation accuracy: 6.42 %
Confusion matrix:
[[ 0.   0.   0. ]
 [69.8 64.2 66. ]
 [ 0.   0.   0. ]]


**Explore the classifier**. How many support vectors are there? What are support vectors?

In [10]:
all_support_vectors = svc.support_vectors_ #Each line = 1 "Support Vector" ; 1024 columns forming a 32x32 image 
vectors_per_class = svc.n_support_ #Number of "Support Vector" for each class

print(len(all_support_vectors[0]))
print(len(vectors_per_class))


256
3


**Try to find the best "C" (error penalty) and "gamma" parameters** using cross-validation. What influence does "C" have on the number of support vectors?

In [2]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

#kf = StratifiedKFold(1, random_state=33)

total_score = {{}}
total_conf = {{}}

C = [2**10, 2**13, 2**14]
GAMMA = [2**-1, 2**0, 2**1]
for c in C:
    for gamma in GAMMA:
        #for train,test in kf.split(dataset.train['hog'], dataset.train['labels']):
        svc = SVC(C=c, kernel='rbf', gamma=gamma )
        train_x, mean, std = normalize(dataset.train['hog'][:14000])
        train_y = dataset.train['labels'][:14000]
        svc.fit(train_x, train_y)

        test_x = (dataset.train['hog'][14000:]-mean)/std
        test_y = dataset.train['labels'][14000:]
        accuracy, conf = evaluate_classifier(svc, test_x, test_y)

        total_score[c][gamma] = accuracy
        total_conf[c][gamma] = conf
    
    

In [5]:
print(evaluate_classifier(svc, train_x, train_y))

(100.0, array([[4651,    0,    0],
       [   0, 4679,    0],
       [   0,    0, 4670]], dtype=int64))


In [14]:
svc = SVC(C=1e5, kernel='rbf', gamma=1./256 )
svc.fit(train_x,train_y)
print(evaluate_classifier(svc,test_x,test_y))

(82.6, array([[310,  45,  14],
       [ 29, 245,  45],
       [ 10,  31, 271]], dtype=int64))


In [4]:
for key in total_score.keys():
    print("Validation accuracy for C value of", key, ": %.2f %%"%total_score[key])
    print("Confusion matrix for C value of", key, ":")
    print(total_conf[key])

Validation accuracy for C value of 1024 : 32.10 %
Confusion matrix for C value of 1024 :
[[  0   0   0]
 [349 321 330]
 [  0   0   0]]
Validation accuracy for C value of 8192 : 32.10 %
Confusion matrix for C value of 8192 :
[[  0   0   0]
 [349 321 330]
 [  0   0   0]]
Validation accuracy for C value of 16384 : 32.10 %
Confusion matrix for C value of 16384 :
[[  0   0   0]
 [349 321 330]
 [  0   0   0]]


**Retrain** your best parameters on all training data. Compare the results to the other classifiers.

In [11]:
2**3

8